<a href="https://colab.research.google.com/github/esuda/mineiracao_dados_complexos/blob/master/Big%20Data/Big_Data_010_Trabalho_2_Sentimento_Noriaki_Suda_Wakabayashi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pacote de dados


*   Abra o link do pacote de dados: https://tinyurl.com/bd10-tweets
*   Selecione a opção "Adicionar Atalho ao Drive"

In [2]:
!pip install mrjob nltk twython

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!ls "/content/drive/My Drive/twitter_cs"

negative-words.txt  sample_clean_larger.csv	twcs_clean.csv
positive-words.txt  sample_clean_larger.csv.gz	twcs_clean.csv.gz
sample_clean.csv    sample.csv			twcs.csv


In [43]:
!cat "/content/drive/My Drive/twitter_cs/sample_clean.csv" | head

tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregarded and the tapped notification under the keyboard is opened😡😡😡,119236,
119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,"@105835 Your business means a lot to us. Please DM your name  zip code and additional details about your concern. ^RR https://t.co/znUu1VJn9r",,119239
119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm sure you won't! Because you don't have to!,119238,
119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,"@105836 LiveChat is online at the moment - https://t.co/SY94VtU8Kq or contact 03331 031 031 option 1  4  3 (Leave a message) to request a call back",119241,119242
119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've tried leaving a voicemail several times in the past week https://t.co/NxVZj

# Ver porque nao esta saindo resultado pelo logger de warning

In [87]:
%%file trab2.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import re
import logging

class Sentiment(MRJob):

  def mapper_init(self):
    nltk.download('vader_lexicon', quiet=True)

    self.sentiment = SentimentIntensityAnalyzer()

  def mapper(self, _, line):
    try:
      tweet_id, author_id, inbound, created_at, text, response_tweet_id, in_response_to_tweet_id = line.split(",")

      author_id = author_id.lower().strip()
      flag_author = bool(re.match("^[0-9]+$", author_id))

      if inbound == "True":
        raise ValueError("Inbound") 
      elif flag_author == True:
        raise ValueError("Author")

      polaridade = self.sentiment.polarity_scores(text)['compound']
      yield author_id, (polaridade, 1)

    except ValueError as error:
      logging.warning(error.args)
      logging.warning("linha descartada: "+line)
      pass

  def combiner(self, key, values):
    try:
      polaridade_sum = 0
      quantidade = 0
      for value in values:
        polaridade_sum += value[0]
        quantidade += value[1]
    
      yield key, (polaridade_sum, quantidade)
    
    except:
      logging.warning("Deu ruim Combiner")     
      pass

  def reducer(self, key, values):
    try:
      polaridade_sum = 0
      quantidade = 0
      for value in values:
        polaridade_sum += value[0]
        quantidade += value[1]

      yield key, str(polaridade_sum/quantidade)

    except:
      logging.warning("Deu ruim reducer")     
      pass

if __name__ == '__main__':
    logging.basicConfig(filename="log.txt")
    Sentiment.run()

Overwriting trab2.py


In [91]:
!python trab2.py -r local "/content/drive/My Drive/twitter_cs/sample_clean.csv" 

No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/trab2.root.20211102.033133.190049
Running step 1 of 1...
job output is in /tmp/trab2.root.20211102.033133.190049/output
Streaming final output from /tmp/trab2.root.20211102.033133.190049/output...
"o2"	"0.7783"
"southwestair"	"0.6114"
"spotifycares"	"0.512325"
"sprintcare"	"0.2263"
"british_airways"	"0.2015"
"chasesupport"	"0.3182"
"comcastcares"	"-0.1217"
"hpsupport"	"0.6369"
"applesupport"	"0.5644153846153845"
"ask_spectrum"	"-0.2732"
"author_id"	"0.0"
"tesco"	"0.48062499999999997"
"upshelp"	"0.2263"
"virgintrains"	"0.171025"
Removing temp directory /tmp/trab2.root.20211102.033133.190049...


In [ ]:
# Sanity Check pra ver se esta agrupando certo
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon', quiet=True)
analyzer = SentimentIntensityAnalyzer()

df = pd.read_csv("/content/drive/My Drive/twitter_cs/sample_clean.csv" )

df["test"] = df.author_id.str.match("^[0-9]+$")

df['compound'] = [analyzer.polarity_scores(x)['compound'] for x in df['text']]

df[df["test"]==False].filter(["author_id", "compound"]).groupby("author_id")["compound"].mean()

In [ ]:
!cat log.txt

In [90]:
!rm log.txt

In [ ]:
!python trab2.py -r local "/content/drive/My Drive/twitter_cs/sample_clean_larger.csv.gz" 

In [ ]:
!python trab2.py -r local "/content/drive/My Drive/twitter_cs/twcs_clean.csv.gz" 

In [22]:
import re
names = ["asdfaser", "1234", "21asdfa", "sdf1234531", ""]
for name in names:
  print(name, "\t", bool(re.match("^[0-9]+$", name)))

asdfaser 	 False
1234 	 True
21asdfa 	 False
sdf1234531 	 False
 	 False
